### Risk Management and Portfolio optimization

##### Algunos conceptos que vamos a ver en esta notebook son:

* Análisis de retornos y de volumen. Algunas conclusiones.
* Distribución de los retornos. Percentiles y quantiles. ¿Qué cosas ver?
* Varianza, desvío estandar (volatilidad) - Indicadores financieros (oscilador de volatilida y radio de volatilidad)
* Correlación, covarianza.
* Introducción a la optimización de portafolios
* Cálculo de medidas de riesgo